# Zillow Prize Challenge

### Import some libraries

In [77]:
import pandas as pd
import numpy as np
import pdb
import seaborn
import math
import seaborn as sns 
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sklearn
from sklearn import preprocessing
import datetime as dt
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import euclidean_distances
import sklearn.metrics.pairwise
import scipy.spatial.distance
from scipy.spatial import distance
from scipy.spatial.distance import pdist, cdist, squareform
from scipy.cluster import hierarchy
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.spatial.distance as ssd
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy import sparse
from sklearn.cluster import DBSCAN

from sklearn.cluster import KMeans
import folium
import random
from matplotlib import colors as mcolors

from geopy.geocoders import Nominatim
from math import sin, cos, sqrt, atan2, radians


%matplotlib inline
warnings.filterwarnings('ignore')


In [94]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

### Read the data from properties, training and submission file(used for training and prediction)

In [2]:
df = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/properties_2017.csv', low_memory=False )
#trainingData = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/train_2016_v2.csv', low_memory=False )
#submission_data = pd.read_csv('/Users/kiranmayi/Desktop/Sem1/DSF_519/Hw2/sample_submission.csv', low_memory=False )

### Identify which features need Mode and Mean to fill in their missing data

In [3]:
featuresWithMode = ['bathroomcnt',
                    'bedroomcnt',
                    'fireplacecnt',
                    'garagecarcnt',
                    'numberofstories',
                    'unitcnt',
                    'yearbuilt',
                    'regionidzip',
                    'latitude',
                    'longitude',
                    'heatingorsystemtypeid',
                    'buildingqualitytypeid',
                    'buildingclasstypeid']

featuresWithMean = ['finishedsquarefeet12',
                    'finishedsquarefeet15',
                    'calculatedfinishedsquarefeet',
                    'structuretaxvaluedollarcnt',
                    'landtaxvaluedollarcnt',
                    'taxamount']

categoricalFeatures = ['heatingorsystemtypeid',
                       'buildingclasstypeid'
                      ]

TotalFeatures = ['parcelid',
                 'bathroomcnt',
                 'bedroomcnt',
                 'fireplacecnt',
                 'garagecarcnt',
                 'numberofstories',
                 'unitcnt',
                 'yearbuilt',
                 'regionidzip',
                 'finishedsquarefeet12',
                 'finishedsquarefeet15',
                 'calculatedfinishedsquarefeet',
                 'structuretaxvaluedollarcnt',
                 'landtaxvaluedollarcnt',
                 'taxamount',
                 'latitude',
                 'longitude',
                 'heatingorsystemtypeid_enc',
                 'buildingqualitytypeid',
                 'buildingclasstypeid_enc']

Importantfeatures = ['parcelid',
                     'latitude',
                     'longitude', 
                     'regionidzip',
                     'finishedsquarefeet15',
                     'calculatedfinishedsquarefeet',
                     'taxamount',
                     'bathroomcnt',
                     'bedroomcnt',
                     'numberofstories',
                     'unitcnt',
                     'yearbuilt',
                     'buildingqualitytypeid',                     
                     'buildingclasstypeid_enc']

FeaturesForSimilarity = ['latitude',
                         'longitude', 
                         'regionidzip',
                         'calculatedfinishedsquarefeet',
                         'finishedsquarefeet15',
                         'numberofstories',
                         'unitcnt',
                         'buildingqualitytypeid',
                         'buildingclasstypeid_enc',
                         'yearbuilt',
                         'taxamount',
                         'bathroomcnt',
                         'bedroomcnt']

FeaturesForDisplay = [   'finishedsquarefeet15',
                         'numberofstories',
                         'unitcnt',
                         'buildingqualitytypeid',
                         'buildingclasstypeid_enc',
                         'yearbuilt',
                         'taxamount',
                         'bathroomcnt',
                         'bedroomcnt',
                         'parcelid',
                         'cluster',
                         'latitude',
                         'longitude']

### Fill NA rows/columns with appropriate Mean or Mode and get the data of interest

In [4]:
cleanedData = deepcopy( df )
for feature in featuresWithMean:
    mean = cleanedData[feature].mean()
    #print("Mean for feature ", feature, "is ", mean.real)
    cleanedData[feature].fillna(mean.real, inplace=True)

for feature in featuresWithMode:
    mode = cleanedData[feature].mode()
    #print("Mode for feature ", feature, "is ", mode.real[0])
    cleanedData[feature].fillna(mode.real[0], inplace=True)
    
for feature in categoricalFeatures:
    #oneHot = pd.get_dummies( cleanedData[feature] )
    encoding = pd.factorize(cleanedData[feature])
    cleanedData = cleanedData.drop( feature, axis=1 )
    cleanedData[ feature + '_enc'] = encoding[0]

DataOfInterest = cleanedData[ TotalFeatures ]

#print(DataOfInterest.head(1))



### Designing a pairwise distance function
#### Euclidean
#### Custom Function

In [5]:

def PairWiseDistanceEuclidean(u, v):
    return np.sqrt(((u-v)**2).sum())

def getDistance( lat_u, lat_v, long_u, long_v ):
    R = 6373.0
    dlon = long_v - long_u
    dlat = lat_v - lat_u
    a = sin(dlat / 2)**2 + cos(lat_u) * cos(lat_v) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    conversion_factor = 0.62137119
    distance = R * c * conversion_factor
    #print(distance)
    return distance

def customePairwise( u, v ):
    latitude_u = radians(u['latitude'] / pow(10,6))
    longtitude_u = radians(u['longitude'] / pow(10,6))
    latitude_v = radians(v['latitude'] / pow(10,6))
    longtitude_v = radians(v['longitude'] / pow(10,6))
    distance = getDistance( latitude_u, latitude_v, longtitude_u, longtitude_v)
    zip_u = u['regionidzip']
    zip_v = v['regionidzip']
    zip_distance = abs( zip_u - zip_v )
    #print(zip_distance)
    sqrtfeet_u = u['calculatedfinishedsquarefeet']
    sqrtfeet_v = v['calculatedfinishedsquarefeet']
    finishedsqrtfeet = ( abs( ( ( u['unitcnt'] * u['numberofstories'] * u['finishedsquarefeet15'] ) - sqrtfeet_u ) - \
                        ( ( v['unitcnt'] * v['numberofstories'] * v['finishedsquarefeet15'] ) - sqrtfeet_v ) ) ) / 100
    #print(finishedsqrtfeet)
    quality = abs( u['buildingqualitytypeid'] - v['buildingqualitytypeid'] ) + \
              abs( u[ 'buildingclasstypeid_enc' ] - v[ 'buildingclasstypeid_enc' ] )
    #print(quality)
    year = abs( u ['yearbuilt'] - v[ 'yearbuilt'] )
    #print(year)
    tax = abs( ( u ['taxamount'] / u['finishedsquarefeet15'] ) - ( v[ 'taxamount'] / v['finishedsquarefeet15'] ) )
    #print(tax)
    rooms = abs( ( u['bedroomcnt'] - v['bedroomcnt']) + ( u['bathroomcnt'] - v['bathroomcnt'] )  )    
    #print(rooms)
    
    return( 0.2*distance + 0.05*zip_distance + 0.15*finishedsqrtfeet + 0.1*quality + 0.1*year + 0.2*tax + 0.1*rooms )


### Compute Eucledian distance and cluster based on them

In [12]:
DistanceSampleDataEuclidean = DataOfInterest[Importantfeatures].head(1000)

distance_euclidean = cdist(DistanceSampleDataEuclidean[FeaturesForSimilarity], DistanceSampleDataEuclidean[FeaturesForSimilarity], PairWiseDistanceEuclidean)
#distArray1 = ssd.squareform(A)

num_clusters = 10

kmeans_eucledian = KMeans(n_clusters=num_clusters, random_state=0).fit(distance_euclidean)
labels = kmeans_eucledian.labels_

DistanceSampleDataEuclidean['cluster'] = kmeans_eucledian.labels_




### Plot the clusters on a map based on location

In [108]:
data = DistanceSampleDataEuclidean[FeaturesForDisplay]
data[['latitude']] = data[['latitude']] / 1000000
data[['longitude']] = data[['longitude']] / 1000000

map = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=12)

for i in range(0, num_clusters):
    locations = data.loc[ data['cluster'] == i ] 
    locationlist = locations[['latitude', 'longitude']].values.tolist()
    r1 = random.randint(0,255)
    r2 = random.randint(0,255)
    r3 = random.randint(0,255)
    color2 = '#%02X%02X%02X' % (r2,r3,r1)
    for point in range(0, len(locationlist)):
        pop_string = "ID:" + str( locations.iloc[point]['parcelid'] ) + "  Stories:" + \
                        str( locations.iloc[point]['numberofstories'] ) + "  Year:" + \
                        str( locations.iloc[point]['yearbuilt'] ) + "  Tax:" + \
                        str( locations.iloc[point]['taxamount'] ) + " SqrFt:" + \
                        str( locations.iloc[point]['finishedsquarefeet15'] ) 
        folium.Marker(locationlist[point],popup=pop_string, 
                      icon=folium.Icon(color=colors[i], icon_color=color2)).add_to(map)
map

### Compute pairwise distance using the custom function defined and use the distance matrix to compute clusters

In [110]:
DistanceSampleData = DataOfInterest[Importantfeatures].head(1000)

distance_custom = []

for i in range(0, len(DistanceSampleData) ):
    temp_dist = []
    for j in range(0, len(DistanceSampleData)):
        temp_dist.append( customePairwise(DistanceSampleData.iloc[i],DistanceSampleData.iloc[j]) )
    distance_custom.append(temp_dist)

kmeans_eucledian = KMeans(n_clusters=num_clusters, random_state=0).fit(distance_custom)
labels = kmeans_eucledian.labels_

DistanceSampleData['cluster'] = kmeans_eucledian.labels_
    

### Plot the clusters on a map based on location

In [111]:
data = DistanceSampleData[FeaturesForDisplay]
data[['latitude']] = data[['latitude']] / 1000000
data[['longitude']] = data[['longitude']] / 1000000

map = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=12)

for i in range(0, num_clusters):
    locations = data.loc[ data['cluster'] == i ] 
    locationlist = locations[['latitude', 'longitude']].values.tolist()
    r1 = random.randint(0,255)
    r2 = random.randint(0,255)
    r3 = random.randint(0,255)
    color2 = '#%02X%02X%02X' % (r2,r3,r1)
    for point in range(0, len(locationlist)):
        pop_string = "ID:" + str( locations.iloc[point]['parcelid'] ) + "  Stories:" + \
                        str( locations.iloc[point]['numberofstories'] ) + "  Year:" + \
                        str( locations.iloc[point]['yearbuilt'] ) + "  Tax:" + \
                        str( locations.iloc[point]['taxamount'] ) + " SqrFt:" + \
                        str( locations.iloc[point]['finishedsquarefeet15'] ) 
        folium.Marker(locationlist[point], popup=pop_string, icon=folium.Icon(color=colors[i], icon_color=color2)).add_to(map)
map

### Use dbscan clustering model.

In [148]:

db = DBSCAN(eps=5, min_samples=3, metric="precomputed")
y_db = db.fit_predict(distance_custom)


In [ ]:
'''Z = hierarchy.linkage(B, 'single')
plt.figure(figsize=(25,10),facecolor='white')
hierarchy.dendrogram(Z)
plt.xticks(rotation=-45)
plt.show()

Z = hierarchy.linkage(D, 'single')
plt.figure(figsize=(25,10),facecolor='white')
dend = hierarchy.dendrogram(Z)
plt.xticks(rotation=-45)
plt.show()
print(len(dend['color_list']))'''

In [ ]:


#dm = pdist(DistanceSampleData, PairWiseDistance)
#print(dm)

dm = pdist(DistanceSampleData, PairWiseDistance1)
print(dm)

Z = hierarchy.linkage(dm, method='single')
plt.figure(figsize=(25,10),facecolor='white')
hierarchy.dendrogram(Z)
plt.xticks(rotation=-45)
plt.show()

assignments = hierarchy.fcluster(hierarchy.linkage(dm, method='complete'),100,criterion='maxclust')
print(assignments)